In [ ]:
import os
import json
import pprint
import copy

os.chdir('../../../src/summaflow/')
os.getcwd()

In [ ]:
# # necessary functions
def deep_merge(d1, d2):
    """
    Recursively merge d2 into d1.
    """
    for key, value in d2.items():
        if key in d1 and isinstance(d1[key], dict) and isinstance(value, dict):
            deep_merge(d1[key], value)  # Recursive merge for nested dictionaries
        else:
            d1[key] = value  # Overwrite or add new key-value pair
    return d1

# CLASS file

In [ ]:
info_block = {
    "vars": {
        "case_name": "MESH for Bow River at Calgary",
        "author": "Kasra Keshavarz",
        "location": "University of Calgary",
    },
    "comments": {
        "case_name": "01 TITLE",
        "author": "02 NAME",
        "location": "03 PLACE"
    },
    "formats": {}, # none means no change
    "columns": {
        "case_name": 3,
        "author": 3,
        "location": 3,
    }, # none means column 2
}

case_block = {
    "vars": {
        "case": {
            "centroid_lat": 45.2,
            "centroid_lon": -114.2,
            "reference_height_wndspd": 40.0,
            "reference_height_spechum_airtemp": 40.0,
            "reference_height_surface_roughness": 50.0,
            "ground_cover_flag": -1.0, # meaning land cover
            "ILW": 1, # MESH,CLASS>3.4 defaults to 1
            "NL": 30, # total number of subbasins
            "NM": 5, # total number of GRUs - 1
        },
    },
    "comments": {
        "case": '04 DEGLAT/DEGLON/ZRFM/ZRFH/ZBLD/GC/ILW/NL/NM'
    },
    "formats": {
        "case": {
            "centroid_lat": 2,
            "centroid_lon": 2,
            "reference_height_wndspd": 1,
            "reference_height_spechum_airtemp": 1,
            "reference_height_surface_roughness": 1,
            "ground_cover_flag": 1,
            "ILW": 0,
            "NL": 0,
            "NM": 0,
        },
    },
    "columns": {
        "case": {
            "centroid_lat": 4,
            "centroid_lon": 12,
            "reference_height_wndspd": 25,
            "reference_height_spechum_airtemp": 35,
            "reference_height_surface_roughness": 45,
            "ground_cover_flag": 52,
            "ILW": 60,
            "NL": 65,
            "NM": 70,
        },
    },
}

gru_block = {
    "vars": [
        {
            "veg": {
                "class": "needleleaf",
            },
            "hyd": {
                "line13": {
                    "mid": "1 Needleleaf Forest (sub-polar)",
                },
            },
        },
        {
            "veg": {
                "class": "needleleaf",
            },
            "hyd": {
                "line13": {
                    "mid": "2 Mostly Needleleaf (tropical)"
                },
            },
        },
        {
            "veg": {
                "class": "broadleaf",
            },
            "hyd": {
                "line13": {
                    "mid": "3 Broadleaf Forest (Taiga)",
                },
            },
        },
        {
            "veg": {
                "class": "crops",
            },
            "hyd": {
                "line13": {
                    "mid": "4 Cropland",
                },
            },
        },
        {
            "veg": {
                "class": "grass",
            },
            "hyd": {
                "line13": {
                    "mid": "5 Grassland-moss",
                },
            },
        },
        {
            "veg": {
                "class": "barrenland",
            },
            "hyd": {
                "line13": {
                    "mid": "6 Barrenland",
                },
            },
        },
    ],
}

In [ ]:
# load the default values for each GRU
with open('./templates/default_CLASS_parameters.json', 'r') as file:
    data = json.load(file)

# populate new dictionary for blocks
populating_list = []

# deep update GRU blocks
for idx, block in enumerate(gru_block['vars']):
    new_data = copy.deepcopy(data)

    # deep merge
    it = deep_merge(new_data['class_defaults'], block)
    
    # update the block dictionary
    populating_list.append(it)

gru_block.update({'vars': populating_list})

# add formats, columns, and comments
new_keys = ['formats', 'comments', 'columns']
for key in new_keys:
    gru_block[key] = data[key]

In [ ]:
pprint.pprint(gru_block)

In [ ]:
from jinja2 import Environment, FileSystemLoader

def raise_helper(msg):
    raise Exception(msg)

environment = Environment(
    loader=FileSystemLoader("templates/"),
    trim_blocks=True,
    lstrip_blocks=True,
    line_comment_prefix='##',
)
environment.globals['raise'] = raise_helper

template = environment.get_template("MESH_parameters_CLASS.ini.jinja")

# create content
content = template.render(
    info_block=info_block,
    case_block=case_block,
    gru_block=gru_block,
    variables="vars",
    comments="comments",
    formats="formats",
    columns="columns",
)

print(content)

____

# Hydrology file

In [ ]:
routing_dict = [
    {
        "r2n": 2.87,
        "flz": 7,
    },
    {
        "r2n": 0.2,
    },
]

gru_dict = [
    {
        'block1': {
            "zsnl": 0.3
        }
    },
    {
        'block2': {
            "zplg": 0.9
        }
    },
]

In [ ]:
# load the default values for each GRU
with open('./templates/default_hydrology_parameters.json', 'r') as file:
    data = json.load(file)

# components
routing_defaults = data.get('routing')
gru_defaults = data.get('gru')

# deep update routing block
for idx, routing_block in enumerate(routing_dict):
    defaults = copy.deepcopy(routing_defaults)

    # deep merge
    defaults.update(routing_block)

    # update the dict (list of dicts)
    routing_dict[idx].update(defaults)

# deep update gru block
for idx, gru_block in enumerate(gru_dict):
    defaults = copy.deepcopy(gru_defaults)

    # update default values
    for k, v in gru_block.items():
        defaults.update(v)

    # update the dict
    gru_dict[idx][k].update(defaults)

In [ ]:
routing_dict

In [ ]:
gru_dict

In [ ]:
# hydrology template file
template_hydrology = environment.get_template("MESH_parameters_hydrology.ini.jinja")

# create content
content_hydrology = template_hydrology.render(
    routing_dict=routing_dict,
    gru_dict=gru_dict,
)

print(content_hydrology)

____

# Input config file

In [ ]:
options_dict = {
    "flags": {
        "forcing": {
            "BASINSHORTWAVEFLAG": "RDRS_v2.1_P_FB_SFC",
            "BASINHUMIDITYFLAG": "RDRS_v2.1_P_HU_1.5m",
            "BASINRAINFLAG": "RDRS_v2.1_A_PR0_SFC",
            "BASINPRESFLAG": "RDRS_v2.1_P_P0_SFC",
            "BASINLONGWAVEFLAG": "RDRS_v2.1_P_FI_SFC",
            "BASINWINDFLAG": "RDRS_v2.1_P_UVC_10m",
            "BASINTEMPERATUREFLAG": "RDRS_v2.1_P_TT_1.5m",
            "BASINFORCINGFLAG": {
                "start_date": 19800101,
                "hf": 15,
            },
        },
        "etc": {
            "PBSMFLAG": "on",
        },
    },
    "outputs": {
        "result": "results",
    },
    "dates": {
        "start_hour": 23,
        "end_day": 352,
        "end_hour": 3
    },
}

In [ ]:
# load the default values for each GRU
with open('./templates/default_input_run_options.json', 'r') as file:
    options = json.load(file)

# deep update the dictionary
options['settings'].update(deep_merge(options['settings'], options_dict))

In [ ]:
pprint.pprint(options)

In [ ]:
# options template file
template_options = environment.get_template("MESH_input_run_options.ini.jinja")

# create content
content_options = template_options.render(
    options_dict = options,
)

print(content_options)